#Prediction for IoT Traffic Simulation

###Author: Neeraj Asthana (under Professor Robert Brunner)

###Collaborators: Anchal, Rishabh, Vaishali

___

##Problem Statement
Recent advances in sensor technology, infrastructure for estimating temperature, pressure, humidity, precipitation, etc. are readily available on roadways to predict traffic states. The goal and novelty of our project is to combine streamed GPS traffic estimation algorithms with dynamic road sensor data to accurately predict traffic states and efficiently advise and direct drivers.

##Time Decay Model for Speed Estimation

Used to estimate Average Road Speeds for specific segments of road by weighting speed observations.

####Observations
* Higher Speeds are more valuable the lower speeds
* Recent Obersvations are favored over historical observations
* Each data point will be of the form ($t_i, v_i, l_i$) or (time, velocity, location)


####Time Weight
An observation's timestamp ($t_i$) is weighted  by: $w(i,t) = \frac{f(t_i - L)}{f(t - L)}$

*f* is some positive, monotonic, non-decreasing function

*L* is some Landmark time (starting time)

*t* is the most recent timestamp

####Velocity Weight
An observation's velocity ($v_i$) is weighted  by: $w^v (i) = g (v_i)$

*g* is some positive, monotonic, non-decreasing function

####Combination of Weights
Weight combinations of the time and velocity weights:

$w^* (i,t) = w(i,t) \cdot w^v (i) = \frac{f(t_i - L)}{f(t - L)} \cdot g(v_i)$

####Aggregating observatoins
Calculates average velocity of a road segment (*r*) by aggregating most recent and historical observations. In order compute these values efficiently and to be able to update values, I will persist 2 quantities in Spark , $X,Y$ which will be then be used to calulate $\overline{V}(r) = \frac{X}{Y}$

$$X = \sum_{n=1}^{m} f(t_i - L) \cdot g(v_i) \cdot v_i$$         

$$Y = \sum_{n=1}^{m} f(t_i - L) \cdot g(v_i)$$
